In [1]:
import os
import sys
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.io import loadmat

In [2]:
import warnings
warnings.simplefilter("ignore")

In [3]:
from pyod.models.pca import PCA
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.lof import LOF
from pyod.models.cblof import CBLOF
from pyod.models.knn import KNN
from pyod.models.hbos import HBOS
from pyod.models.abod import ABOD
from pyod.models.iforest import IForest
from pyod.models.feature_bagging import FeatureBagging

In [4]:
from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score

In [5]:
mat_file_list=['arrhythmia.mat','cardio.mat','glass.mat','ionosphere.mat','letter.mat','lympho.mat','mnist.mat','musk.mat','optdigits.mat','pendigits.mat','pima.mat','satellite.mat','satimage-2.mat','shuttle.mat','vertebral.mat','vowels.mat','wbc.mat']

In [6]:
mat_file_list

['arrhythmia.mat',
 'cardio.mat',
 'glass.mat',
 'ionosphere.mat',
 'letter.mat',
 'lympho.mat',
 'mnist.mat',
 'musk.mat',
 'optdigits.mat',
 'pendigits.mat',
 'pima.mat',
 'satellite.mat',
 'satimage-2.mat',
 'shuttle.mat',
 'vertebral.mat',
 'vowels.mat',
 'wbc.mat']

In [7]:
df_columns=['Data','#Sample','#Dimensions','Outlier Perc','PCA','MCD','OCSVM','LOF','CBLOF','KNN','HBOS','ABOD','IFOREST','FEATUREBAGGING']

In [8]:
df_columns

['Data',
 '#Sample',
 '#Dimensions',
 'Outlier Perc',
 'PCA',
 'MCD',
 'OCSVM',
 'LOF',
 'CBLOF',
 'KNN',
 'HBOS',
 'ABOD',
 'IFOREST',
 'FEATUREBAGGING']

In [9]:
roc_df=pd.DataFrame(columns=df_columns)
prn_df=pd.DataFrame(columns=df_columns)
time_df=pd.DataFrame(columns=df_columns)
print(roc_df,prn_df,time_df)

Empty DataFrame
Columns: [Data, #Sample, #Dimensions, Outlier Perc, PCA, MCD, OCSVM, LOF, CBLOF, KNN, HBOS, ABOD, IFOREST, FEATUREBAGGING]
Index: [] Empty DataFrame
Columns: [Data, #Sample, #Dimensions, Outlier Perc, PCA, MCD, OCSVM, LOF, CBLOF, KNN, HBOS, ABOD, IFOREST, FEATUREBAGGING]
Index: [] Empty DataFrame
Columns: [Data, #Sample, #Dimensions, Outlier Perc, PCA, MCD, OCSVM, LOF, CBLOF, KNN, HBOS, ABOD, IFOREST, FEATUREBAGGING]
Index: []


In [10]:
from time import time
random_state = np.random.RandomState(42)
for mat_file in mat_file_list:
    print("\n... Processing",mat_file,'.....')
    mat = loadmat(os.path.join('Downloads',mat_file))
    
    X = mat['X']
    y = mat['y'].ravel()
    
    outliers_fraction = np.count_nonzero(y) / len(y)
    
    outliers_percentage = round(outliers_fraction * 100, ndigits=4)
    
    roc_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    prn_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    time_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state = random_state)
    
    X_train_norm, X_test_norm = standardizer(X_train, X_test)
    
    classifiers = {'Angle-based Outlier Detector (ABOD)' : ABOD(contamination=outliers_fraction), 
                  'Cluster-based Local Outlier Factor': CBLOF(contamination=outliers_fraction,check_estimator=False ,random_state=random_state),
                  'Feature Bagging': FeatureBagging(contamination=outliers_fraction, random_state=random_state),
                  'Historgram-base Outlier Detection (HBOS)': HBOS(contamination=outliers_fraction),
                  'Isolation Forest' : IForest(contamination=outliers_fraction, random_state=random_state),
                  'K Nearest Neighbors (KNN)': KNN(contamination=outliers_fraction),
                  'Local Outlier Factor (LOF)' : LOF(contamination=outliers_fraction),
                  'Minimum Covariance Determinant (MCD)': MCD(contamination=outliers_fraction, random_state=random_state),
                  'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction),
                  'Principal Component Analysis (PCA)': PCA(contamination=outliers_fraction, random_state=random_state)}
    for clf_name, clf in classifiers.items():
        t0 = time()
        clf.fit(X_train_norm)
        test_scores = clf.decision_function(X_test_norm)
        t1 = time()
        duration = round(t1-t0, 4)
        time_list.append(duration)
        
        roc = round(roc_auc_score(y_test, test_scores), 4)
        prn = round(precision_n_scores(y_test, test_scores), 4)
        print(f"{clf_name} ROC: {roc}, precision @ rank n: {prn}, execution time: {duration}s' ")
        
        roc_list.append(roc)
        prn_list.append(prn)
        
    temp_df = pd.DataFrame(time_list).transpose()
    temp_df.columns = df_columns
    time_df = pd.concat([time_df, temp_df], axis=0)
    
    temp_df = pd.DataFrame(roc_list).transpose()
    temp_df.columns = df_columns
    roc_df = pd.concat([roc_df, temp_df], axis=0)
    
    temp_df = pd.DataFrame(prn_list).transpose()
    temp_df.columns = df_columns
    prn_df = pd.concat([prn_df, temp_df], axis=0)


... Processing arrhythmia.mat .....
Angle-based Outlier Detector (ABOD) ROC: 0.7687, precision @ rank n: 0.3571, execution time: 1.658s' 
Cluster-based Local Outlier Factor ROC: 0.7789, precision @ rank n: 0.4643, execution time: 2.3417s' 
Feature Bagging ROC: 0.7796, precision @ rank n: 0.4643, execution time: 0.9393s' 
Historgram-base Outlier Detection (HBOS) ROC: 0.8511, precision @ rank n: 0.5714, execution time: 2.0306s' 
Isolation Forest ROC: 0.8637, precision @ rank n: 0.6071, execution time: 0.3887s' 
K Nearest Neighbors (KNN) ROC: 0.782, precision @ rank n: 0.5, execution time: 0.0781s' 
Local Outlier Factor (LOF) ROC: 0.7787, precision @ rank n: 0.4643, execution time: 0.0625s' 
Minimum Covariance Determinant (MCD) ROC: 0.8228, precision @ rank n: 0.4286, execution time: 0.7559s' 
One-class SVM (OCSVM) ROC: 0.7986, precision @ rank n: 0.5, execution time: 0.0597s' 
Principal Component Analysis (PCA) ROC: 0.7997, precision @ rank n: 0.5, execution time: 0.0625s' 

... Process

In [11]:
roc_df

,Data,#Sample,#Dimensions,Outlier Perc,PCA,MCD,OCSVM,LOF,CBLOF,KNN,HBOS,ABOD,IFOREST,FEATUREBAGGING
0,arrhythmia,452,274,14.6018,0.7687,0.7789,0.7796,0.8511,0.8637,0.782,0.7787,0.8228,0.7986,0.7997
0,cardio,1831,21,9.6122,0.5892,0.8845,0.6385,0.8373,0.951,0.734,0.588,0.8534,0.9478,0.9616
0,glass,214,9,4.2056,0.6951,0.811,0.7073,0.7073,0.7134,0.8384,0.7043,0.8293,0.6585,0.686
0,ionosphere,351,33,35.8974,0.9181,0.9176,0.9303,0.6052,0.8516,0.932,0.9227,0.9669,0.8257,0.7941
0,letter,1600,32,6.25,0.8783,0.7783,0.8947,0.6063,0.6279,0.8573,0.8765,0.8061,0.5927,0.5216
0,lympho,148,18,4.0541,0.9831,1,1,1,1,1,1,1,1,1
0,mnist,7603,100,9.2069,0.7628,0.8389,0.7157,0.5766,0.7915,0.8498,0.7195,0.8713,0.854,0.8534
0,musk,3062,166,3.1679,0.2161,1,0.473,0.9999,1,0.8009,0.4629,1,1,1
0,optdigits,5216,64,2.8758,0.4894,0.7901,0.5062,0.8774,0.686,0.406,0.5277,0.3822,0.5171,0.526
0,pendigits,6870,16,2.2707,0.667,0.8082,0.4889,0.9348,0.939,0.7371,0.4965,0.8204,0.9235,0.9309


In [12]:
prn_df

,Data,#Sample,#Dimensions,Outlier Perc,PCA,MCD,OCSVM,LOF,CBLOF,KNN,HBOS,ABOD,IFOREST,FEATUREBAGGING
0,arrhythmia,452,274,14.6018,0.3571,0.4643,0.4643,0.5714,0.6071,0.5,0.4643,0.4286,0.5,0.5
0,cardio,1831,21,9.6122,0.1918,0.4932,0.1781,0.4521,0.6027,0.3562,0.1507,0.411,0.5342,0.6849
0,glass,214,9,4.2056,0.25,0.25,0.25,0,0.25,0.25,0.25,0,0.25,0.25
0,ionosphere,351,33,35.8974,0.8431,0.8039,0.8039,0.3922,0.6078,0.8824,0.7843,0.8627,0.6863,0.5686
0,letter,1600,32,6.25,0.4375,0.1875,0.4062,0.0938,0.0625,0.3125,0.3438,0.1875,0.125,0.125
0,lympho,148,18,4.0541,0,1,1,1,1,1,1,1,1,1
0,mnist,7603,100,9.2069,0.3367,0.3912,0.3741,0.1361,0.2687,0.432,0.3673,0.2653,0.3946,0.3878
0,musk,3062,166,3.1679,0.1,1,0.125,0.975,1,0.175,0.125,1,1,1
0,optdigits,5216,64,2.8758,0.0152,0,0.0303,0.2121,0.0303,0,0.0303,0,0,0
0,pendigits,6870,16,2.2707,0.0526,0.1579,0.0526,0.2632,0.3333,0.0702,0.0702,0.0877,0.3158,0.3158


In [13]:
time_df

,Data,#Sample,#Dimensions,Outlier Perc,PCA,MCD,OCSVM,LOF,CBLOF,KNN,HBOS,ABOD,IFOREST,FEATUREBAGGING
0,arrhythmia,452,274,14.6018,1.658,2.3417,0.9393,2.0306,0.3887,0.0781,0.0625,0.7559,0.0597,0.0625
0,cardio,1831,21,9.6122,0.5217,0.1562,0.9072,0.0156,0.3594,0.1562,0.107,0.9018,0.1094,0.0156
0,glass,214,9,4.2056,0.0625,0.0934,0.0313,0,0.2227,0.0156,0,0.0469,0,0
0,ionosphere,351,33,35.8974,0.0937,0.0693,0.0937,0.0156,0.2514,0.0156,0.0156,0.0937,0,0
0,letter,1600,32,6.25,0.4458,0.1719,0.8368,0.0008,0.3624,0.1562,0.0977,1.3851,0.1093,0
0,lympho,148,18,4.0541,0.0312,0.0469,0.0477,0,0.344,0,0,0.0781,0,0.0156
0,mnist,7603,100,9.2069,8.1503,1.3459,52.7916,0.0625,2.3117,7.3618,6.595,3.8283,4.8281,0.1731
0,musk,3062,166,3.1679,2.3691,0.4457,13.3931,0.0625,1.7346,1.9457,1.7427,14.8153,1.1643,0.1562
0,optdigits,5216,64,2.8758,2.9312,0.6762,14.4056,0.0312,1.1409,2.2186,2.3074,1.5535,1.4626,0.0625
0,pendigits,6870,16,2.2707,2.1634,0.3419,5.0813,0.0156,0.8377,0.6651,0.7595,3.4422,1.2217,0.0156


In [14]:
n_samples = len(y)
clusters_separation = [0]

n_inliers = int((1. - outliers_fraction) * n_samples)
n_outliers = int(outliers_fraction * n_samples)
ground_truth = np.zeros(n_samples, dtype=int)
ground_truth[-n_outliers:] = 1

In [15]:
print('Number of inliers: %i' % n_inliers)
print('Number of outliers: %i' % n_outliers)
print('Ground truth shape is {shape}. Outlier are 1 and inliers are 0.\n'.format(shape=ground_truth.shape))
print(ground_truth)

Number of inliers: 357
Number of outliers: 21
Ground truth shape is (378,). Outlier are 1 and inliers are 0.

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1]


In [16]:
for i, clf in enumerate(classifiers.keys()):
    print('Model', i + 1, clf)

Model 1 Angle-based Outlier Detector (ABOD)
Model 2 Cluster-based Local Outlier Factor
Model 3 Feature Bagging
Model 4 Historgram-base Outlier Detection (HBOS)
Model 5 Isolation Forest
Model 6 K Nearest Neighbors (KNN)
Model 7 Local Outlier Factor (LOF)
Model 8 Minimum Covariance Determinant (MCD)
Model 9 One-class SVM (OCSVM)
Model 10 Principal Component Analysis (PCA)
